# Running a Decision Tree on A Larger Dataset

In [ ]:
from SEData.data import *
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [ ]:
question_links = sample_question_links #populate_question_links()
train_links = []
for i in range(1,10):
    new_links = populate_stepback_links(question_links, step_back = i)
    train_links += new_links
train_links += question_links
assert len(train_links) == 500


In [ ]:
test_links = []
for i in range(300,310):
    new_links = populate_stepback_links(question_links, step_back = i)
    test_links += new_links
assert len(test_links) == 500


In [ ]:
len(test_links)

In [ ]:
test_links = [link for link in test_links if link not in train_links]
assert len([link for link in test_links if link in train_links]) == 0

In [ ]:
train_data = [get_text(link) for link in train_links]

In [ ]:
train_data = pd.DataFrame(train_data)
train_data.dropna(axis = 0, how = 'any', inplace = True) #Drop NAs in place. 


In [ ]:
train_labels = data[0]
train_corpus = data[1]

In [ ]:
tfidf = TfidfVectorizer(stop_words = 'english', max_features = 500) #Limit number of features at 500 words with highest Tfidf score
train_matrix = tfidf.fit_transform(train_corpus)

In [ ]:
test_data = [get_text(link) for link in test_links]
test_data = pd.DataFrame(test_data)
test_data.dropna(axis = 0, how = 'any', inplace = True)
test_labels = test_data[0]
test_corpus = test_data[1]

test_matrix = tfidf.transform(test_corpus)


## Train a Decision Tree Model
### X_train : train_matrix
### y_train : train_labels
### X_test : test_matrix
### y_test : test_labels

In [ ]:
tree = DecisionTreeClassifier()
tree.fit(train_matrix, train_labels)

pred_labels = tree.predict(test_matrix)

In [ ]:
print(accuracy_score(test_labels, pred_labels))

In [ ]:
pd.DataFrame({"50 Test Data Labels":test_labels[:50], "50 Predicted Data Labels":pred_labels[:50]})